# Duality

## When do we choose dual problem and what are the advantages in optimization.

In mathematics, if there exists a standard primal form with an objective function, constraints and decision variables, then we can have a dual form based on the information of primal form. The dual form has an inverse objective from the original primal with each variable in the primal becomes a constraint in the dual, and each constraint in the primal becomes a variable in the dual. Below is the basic structures of a primal-dual pair.

Primal Problem
$$
\begin{array}{rc}
\text{maximize:} & \mathbf{c}^T \mathbf{x} \\
\text{subject to:} & A \mathbf{x} \leq \mathbf{b} \\
& \mathbf{x} \geq 0
\end{array}
$$

Dual Problem
$$
\begin{array}{rc}
\text{minimize:} & \mathbf{b}^T \mathbf{y} \\
\text{subject to:} & A^T \mathbf{y} \geq \mathbf{c} \\
& \mathbf{y} \geq 0
\end{array}
$$

$A$ is an $m \times n$ matrix, $\mathbf{b} \in \mathbb{R}^m$ and $\mathbf{c} \in \mathbb{R}^n$

Then why should we form primal-dual pairs and what are the advantages? We are going to talk about three advantages of duality with simple examples illustrated.

### Duality helps to avoid initialization.

While for questions related to infeasiblity, sometimes adding a dual to the primal problem can easily solve it. If we don't build a primal-dual pair and insist on solving the problem with the original form, we have to use initialization. Initialization will have two phases and there will be lots of calculations. To solve such problems efficiently, we can consider adding a dual to the primal problem.

Let's say we have a matrix.

$$
\mathbf{A} = \begin{bmatrix} 4 & 4 \\ 2 & 1 \\ 3 & 2 \\\end{bmatrix}
\hspace{1in}
\mathbf{b} = \begin{bmatrix} 12 \\ -3 \\ 4 \end{bmatrix}
\hspace{1in}
\mathbf{c} = \begin{bmatrix} 2 \\ 3 \end{bmatrix}
$$

In [2]:
import numpy as np
import scipy.linalg as la
from scipy.optimize import linprog

In [3]:
def tableau(c,A,b):
    m,n = A.shape
    I = np.eye(m)
    T = np.vstack([ np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
    return T

def pivot(T,k,l):
    E = np.eye(T.shape[0])
    E[:,l] = -T[:,k]/T[l,k]
    E[l,l] = 1/T[l,k]
    return E@T

In [4]:
A = np.array([[4.,4.],[2.,1.],[3.,2.]])
m,n = A.shape
I = np.eye(m)
c = np.array([2.,3.])
b = np.array([12.,-3.,4.])
T = np.vstack([np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

[[ 4.  4.  1.  0.  0. 12.]
 [ 2.  1.  0.  1.  0. -3.]
 [ 3.  2.  0.  0.  1.  4.]
 [ 2.  3.  0.  0.  0.  0.]]


The matrix shown is not good because we can see a negative in the constraints. When we add slack variables, it will required us to treat this problem as an initialization first by adding a scalar to the objective function. With duality, we can deal with such situation in an efficient way.

In [5]:
A = np.array([[4.,2.,3.],[4.,1.,2.]])
m,n = A.shape
I = np.eye(m)
b = np.array([2.,3.])
c = np.array([12.,-3.,4.])
T = np.vstack([np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

[[ 4.  2.  3.  1.  0.  2.]
 [ 4.  1.  2.  0.  1.  3.]
 [12. -3.  4.  0.  0.  0.]]


When we transfer the original infeasible primal problem to dual problem, we successfully elliminate the negative constraints. Thus we can use simplex method to solve the above matrix and get the solution. After solving the dual, substitute the values of slack variable into the primal and get the final answer.

Now considering multiple-constraints primal problem, it will be super difficult to solve if many infeasible constraints are shown in the primal problem. We don't need to do initalization for the primal as it will be super time consuming. Instead, we can simply build a dual problem based on the primal and calculate priaml-dual pair to get the solution efficiently. If all dual constraints are feasible, we only need to perforrm simplex method to the dual.

### Duality introduces shadow price for primal constraints.
While in reality, we encounter many profit maximization problem. If we aim to maximize the profits of many resource constraints $i$, then it will be useful to build a dual system. The $y_i$ of the corresponding y value helps to observe and adjust maximum profits easily because when $y_i$ increases, the corresponding $i$ product increases its profits. If we would like to add additional units for the resources, then we only need to calculate the dual prices.

For example, a car-made company offers two types of cars A and B, the goal is to maximize the profits. Metal resources and engine assembly are required for the manufacture of a car. We restricted the total amount of cars to 50 and the profits of making A and B are 3 and 2 respectively. The cost of metal resources per unit is 4 and the cost of engine assembly is 5 and the restriction of cost is 100. Now we can build the primal problem.

In symbols, we aim to
$$
\begin{array}{rc}
\text{maximize:} & \mathbf{3A} + \mathbf{2B} \\
\text{subject to:} &  \mathbf{4 metal} + \mathbf{3 engine} \leq \mathbf{100} \\ 
& \mathbf{ A} + \mathbf{ B} \leq \mathbf{50} \\
& \mathbf{A,B,metal,engine} \geq \mathbf{0}
\end{array}
$$


In [6]:
A = np.array([[4.,3.],[1.,1.]])
m,n = A.shape
I = np.eye(m)
b = np.array([100.,50.])
c = np.array([3.,2.])
T = np.vstack([np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

[[  4.   3.   1.   0. 100.]
 [  1.   1.   0.   1.  50.]
 [  3.   2.   0.   0.   0.]]


From the above equations we can only know the profits we are going to make without looking at the marginal costs of products. In reality, we often do not stick with certain resources and fixed types of products. While building a dual problem helps to solve this problem. 


In [7]:
Tdual = tableau(-b,-A.T,-c)
print(Tdual)

[[  -4.   -1.    1.    0.   -3.]
 [  -3.   -1.    0.    1.   -2.]
 [-100.  -50.    0.    0.    0.]]


The signs are negative now due to the change of maximum to minimum while switching primal to dual. Regardless the sign of this particular matrix, if we don't perform in this way, the constraints of the dual will always be feasible. The reason is because the costs cannot be negative for purchasing products. In conclusion, building up such primal-dual system helps to track multiple features for maximum profits problems in reality.

### Duality helps sensitive analysis and models.
For primal problems, changing the constraints for the objective function or adding new constraints could make the original optimal primal solution infeasible. If we build up a primal-dual system, adding a new constraint only changes the objective function by adding one more variable. According to the strong duality theorm, suppose the original primal problem has a feasible solution, then the corresponding dual problem is feasible as well.

Let's perform this situation using a similar matrix as the first point.

In [1]:
A = np.array([[4.,4.],[2.,1.],[3.,2.]])
m,n = A.shape
I = np.eye(m)
c = np.array([2.,3.])
b = np.array([12.,3.,4.])
T = np.vstack([np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

sol = linprog(-c,A,b)
sol.x
sol.message

[[ 4.  4.  1.  0.  0. 12.]
 [ 2.  1.  0.  1.  0.  3.]
 [ 3.  2.  0.  0.  1.  4.]
 [ 2.  3.  0.  0.  0.  0.]]


'Optimization terminated successfully. (HiGHS Status 7: Optimal)'

In [ ]:
A = np.array([[4.,4.],[2.,1.],[3.,2.],[2.,2.]])
m,n = A.shape
I = np.eye(m)
c = np.array([2.,3.])
b = np.array([12.,3.,4.,-1.])
T = np.vstack([np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

sol = linprog(-c,A,b)
sol.x
sol.message

[[ 4.  4.  1.  0.  0.  0. 12.]
 [ 2.  1.  0.  1.  0.  0.  3.]
 [ 3.  2.  0.  0.  1.  0.  4.]
 [ 2.  2.  0.  0.  0.  1. -1.]
 [ 2.  3.  0.  0.  0.  0.  0.]]


"The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is b'At lower/fixed bound')"

The original primial is feasible but it shows infeasible when we add on more constraint. Now we switch primal to dual and thus it will lead us a feasible solution. 

In [11]:
Tdual = tableau(-b,-A.T,-c)
sol = linprog(c,-A,-b)
sol.x
sol.message

'Optimization terminated successfully. (HiGHS Status 7: Optimal)'

Building dual problem for sensitive analysis often solve problems for infeasible primal like the above example illustrated.

### Summary
The above three advantages of duality can be applied to many linear programming problems, especially when primal problems encounter infeasibility or there are less variable numbers but multiple constraints. The duality brings efficiency and convinences when we are working on those linear programming problems. There are also many advantages of duality in transportation optimization, matrix game and network problems as well. Although duality is not always the first-choice method for solving linear programming problems, its functionality is still board in these fields.

## References

Agarwal, L. (2022, February 22). Duality in linear programming, characteristics and advantages of duality. Prinsli.com. Retrieved December 9, 2022, from https://prinsli.com/duality-in-linear-programming/ 

Wikimedia Foundation. (2022, August 13). Duality. Wikipedia. Retrieved December 9, 2022, from https://en.wikipedia.org/wiki/Duality 